# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2025-12-23 16:52:46] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2025-12-23 16:52:46] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2025-12-23 16:52:46] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-12-23 16:52:48] WARNING server_args.py:1529: Attention backend not specified. Use fa3 backend by default.


[2025-12-23 16:52:48] INFO server_args.py:2405: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=48 avail_mem=12.98 GB):  50%|█████     | 10/20 [00:00<00:00, 19.71it/s]

Capturing batches (bs=32 avail_mem=12.97 GB):  50%|█████     | 10/20 [00:00<00:00, 19.71it/s]

Capturing batches (bs=16 avail_mem=12.94 GB):  65%|██████▌   | 13/20 [00:01<00:00, 12.11it/s]

Capturing batches (bs=2 avail_mem=12.92 GB):  85%|████████▌ | 17/20 [00:01<00:00, 10.66it/s]

Capturing batches (bs=1 avail_mem=12.90 GB): 100%|██████████| 20/20 [00:01<00:00, 12.50it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Sanya and I'm a student from the UK.
What are the main things I should consider when shopping in Malaysia?
It depends on the shopping center, but in general, the main things you should consider when shopping in Malaysia are:
- Price
- Quality
- Variety
- Size
- Color
- Packaging
- Discount
- Season
- Location
- Reviews
The first thing you should consider is the price. In general, it is advisable to spend more money on high-end brands or items that are produced in high-quality materials, and smaller items that are priced lower. If you are shopping for something on sale,
Prompt: The president of the United States is
Generated text:  a man. The United States of America is a country. So, the president of the United States is a country. (i)

If the above sentence is true, what can we conclude about the word "country"?
To determine what we can conclude about the word "country" based on the given sentence, let's analyze the sentence step by step:

1.

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name], and I'm a [Age] year old [Occupation]. I'm a [Skill] who enjoys [Favorite Activity]. I'm passionate about [Why I'm Passionate]. I'm always looking for new challenges and opportunities to grow and learn. I'm a [Skill] who is always [What I'm Good At]. I'm [What I Do] and I'm [What I'm Looking Forward To]. I'm excited to meet you and learn more about you. [Name] [Age] [Occupation] [Skill] [Favorite Activity] [Why I'm Passionate] [Why I'm Good

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, also known as "La Ville-Marie" or "La Ville de Paris" and is the largest city in Europe by population. It is located in the Île-de-France region and is the seat of government, administration, and culture for the French Republic. Paris is known for its rich history, art, and cuisine, and is a major tourist destination. The city is also home to many famous landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. Paris is a vibrant and diverse city with a rich cultural heritage that continues to thrive today. The city is also home to many

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by a number of trends that are expected to shape the technology's direction and impact on society. Here are some of the most likely trends that are expected to shape the future of AI:

1. Increased use of AI in healthcare: AI is already being used in healthcare to improve patient outcomes and reduce costs. As AI technology continues to improve, we can expect to see even more widespread use of AI in healthcare, particularly in areas such as diagnosis, treatment planning, and patient care.

2. Greater integration of AI into everyday life: AI is already being integrated into everyday life through the use of voice assistants, such as Siri



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [insert name] and I’m a [insert occupation or profession]. I’ve always been passionate about [insert something that relates to the character]. I love exploring new places, learning new things, and discovering new hobbies. I’m also an avid reader and enjoy writing short stories, poetry, and sometimes a little bit of autobiography. I love travel and exploring new cultures and languages. I am always looking for new experiences and ideas to help me grow and discover my own unique voice. I’m excited to share my journey and experiences with you. [Insert your name] [insert short self-introduction] [Insert your name] [insert short

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, which is located in the south of the country.

Sure, Paris is the capital of France. It's located in the south of the country, near th

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

insert

 your

 name

...

 and

 patient

 monitoring

.



3

In [6]:
llm.shutdown()